In [ ]:
# Import required packages
import os
import shutil

import numpy as np
import sklearn.utils as sku

import Config as conf
import CSV as csv

In [ ]:
# Set LOG_DIR & OUTPUT_DIR
LOG_DIR = conf.LOG_DIR.format('DLSTM')
OUTPUT_DIR = conf.OUTPUT_DIR.format('DLSTM')

# Import CSV data
csi, label, size = csv.getWindows()

In [ ]:
# Import Keras
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.callbacks as kc
import tensorflow.keras.layers as kl
import tensorflow.keras.models as km
import tensorflow.keras.optimizers as ko
import tensorflow.keras.utils as ku

# Set CUDA (use what gpu?) -- comment this if use all GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

# Print tensorflow version
print("Tensorflow:", tf.__version__)
print("Keras:", keras.__version__)

In [ ]:
# Setup Keras LSTM Model
model = None
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    adam = ko.Adam(learning_rate=conf.LEARNING_RATE, amsgrad=True)
    lstm1 = kl.LSTM(
        256,
        unit_forget_bias=True,
        return_sequences=True,
        input_shape=(size[0], size[1]))
    lstm1.add_loss(lambda: 1e-8)
    lstm2 = kl.LSTM(
        64,
        unit_forget_bias=True
    )
    lstm2.add_loss(lambda: 1e-8)
    model = km.Sequential()
    model.add(lstm1)
    model.add(lstm2)
    model.add(kl.Dense(conf.ACTION_CNT, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=["accuracy"]
    )
    model.summary()

In [ ]:
# Check output directory and prepare tensorboard
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)
if os.path.exists(LOG_DIR):
    shutil.rmtree(LOG_DIR)
os.makedirs(LOG_DIR)
tensorboard = kc.TensorBoard(
    log_dir=LOG_DIR,
    write_graph=True,
    write_images=True,
    update_freq=10)
print(
    "Your tensorboard command is:"
)
print("    tensorboard --logdir=" + LOG_DIR)
print("Keras checkpoints and final result will be saved in here:")
print("    " + OUTPUT_DIR)

In [ ]:
# Run KFold
xx, yy = sku.shuffle(csi, label, random_state=0)
for i in range(conf.KFOLD):
    # Roll the data
    xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = xx[int(len(xx) / conf.KFOLD):]
    yTrain = yy[int(len(yy) / conf.KFOLD):]
    xEval = xx[:int(len(xx) / conf.KFOLD)]
    yEval = yy[:int(len(yy) / conf.KFOLD)]

    # If there exists only one action, convert Y to binary form
    if yEval.shape[1] == 1:
        yTrain = ku.to_categorical(yTrain)
        yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(OUTPUT_DIR + "K" + str(i + 1) + "_A{val_accuracy:.6f}_L{val_loss:.6f}.h5")

    # Fit model (learn)
    print(str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) + " th.")
    model.fit(
        xTrain,
        yTrain,
        epochs=conf.EPOCH_CNT,
        batch_size=conf.BATCH_SIZE,
        shuffle=True,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval),
        validation_freq=1,
        use_multiprocessing=True)
print("Epoch completed!")

In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(OUTPUT_DIR + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(OUTPUT_DIR + "model.json", "w") as json:
    json.write(modelJSON)
model.save(OUTPUT_DIR + "model.h5")
print('Model saved!')

In [ ]:
# Finished